<a href="https://colab.research.google.com/github/Thecaiobr/Inteligencia-artifical/blob/main/BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn import preprocessing
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import classification_report
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

In [33]:
colnames=['age','menopause','tumor-size',	'inv-nodes','node-caps','deg-malig','breast','breast-quad','irradiat','Class']
dataset=pd.read_csv('/content/sample_data/breast-cancer.csv', delimiter = "," ,encoding = "ISO-8859-1", names=colnames, header=0)
dataset

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,40-49,premeno,15-19,0-2,yes,3,right,left_up,no,recurrence-events
1,50-59,ge40,15-19,0-2,no,1,right,central,no,no-recurrence-events
2,50-59,ge40,35-39,0-2,no,2,left,left_low,no,recurrence-events
3,40-49,premeno,35-39,0-2,yes,3,right,left_low,yes,no-recurrence-events
4,40-49,premeno,30-34,3-5,yes,2,left,right_up,no,recurrence-events
...,...,...,...,...,...,...,...,...,...,...
281,50-59,ge40,30-34,6-8,yes,2,left,left_low,no,no-recurrence-events
282,50-59,premeno,25-29,3-5,yes,2,left,left_low,yes,no-recurrence-events
283,30-39,premeno,30-34,6-8,yes,2,right,right_up,no,no-recurrence-events
284,50-59,premeno,15-19,0-2,no,2,right,left_low,no,no-recurrence-events


In [34]:

label_encoder = LabelEncoder() #binariza atributos ordinais

input = dataset.drop('Class',axis='columns')
sas = dataset['Class']
target = sas.values

removeRepetidos = lambda array:[x.replace("OneHot__","") for x in array]

In [35]:
#Binarizando os dados ordinais
input['age'] = label_encoder.fit_transform(input['age'])
input['menopause'] = label_encoder.fit_transform(input['menopause'])
input['tumor-size'] = label_encoder.fit_transform(input['tumor-size'])
input['inv-nodes'] = label_encoder.fit_transform(input['inv-nodes'])
input['node-caps'] = label_encoder.fit_transform(input['node-caps'])
input['deg-malig'] = label_encoder.fit_transform(input['deg-malig'])
input['irradiat'] = label_encoder.fit_transform(input['irradiat'])
input['age'] = label_encoder.fit_transform(input['age'])

In [36]:
#Binarizando os dados não ordinais
onehotencoder_rest = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), ['breast', 'breast-quad'])], remainder='passthrough')
input = onehotencoder_rest.fit_transform(input)
names = onehotencoder_rest.get_feature_names()
nomes_atributos = removeRepetidos(names)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Decision tree

In [37]:
#feature_cols = ['age',	'menopause',	'tumor-size',	'inv-nodes',	'node-caps',	'deg-malig'	,'breast',	'breast-quad',	'irradiat']
x = input                               #contains the attribute 
y = sas
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
classifier = DecisionTreeClassifier(criterion="entropy", random_state=100)     # create a classifier object
classifier.fit(x_train, y_train)
DecisionTreeClassifier(criterion='entropy', random_state=100)
y_pred= classifier.predict(x_test)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_test, y_pred)

0.6896551724137931

In [38]:
print(classification_report(y_test, y_pred))

                      precision    recall  f1-score   support

no-recurrence-events       0.82      0.69      0.75        39
   recurrence-events       0.52      0.68      0.59        19

            accuracy                           0.69        58
           macro avg       0.67      0.69      0.67        58
        weighted avg       0.72      0.69      0.70        58



# Naive bayes

In [45]:
#Preparar dados para serem divididos entre treinamento e teste
input = input[:, 1:] # remover essa linha caso a primeira coluna corresponda a dados relevantes

# Indentificando os rótulos das classes
labels = []
for line in range(input.shape[0]):
  if(labels.count(input[line, input.shape[1]-1])==0):
    labels.append(input[line, input.shape[1]-1])

y = np.array(input[:, input.shape[1]-1])
x = (input[:, :(input.shape[1]-1)]).astype(np.float32)

# Gerando os conjuntos de treinamento e teste (validação)
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.25) # 0.25 dos dados no conjunto de teste

In [46]:
#Ajuste do modelo para classes com distribuição gaussiana
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

model.fit(train_x, train_y)

GaussianNB()

In [52]:
#Printando as métricas de avaliação
train_est_y = model.predict(train_x)

print(classification_report(train_y, train_est_y)) # mostra relatório
print('A acurácia é ',accuracy_score(train_est_y, train_y)) # exibe acurácia

              precision    recall  f1-score   support

         0.0       0.87      0.84      0.85       157
         1.0       0.60      0.65      0.62        57

    accuracy                           0.79       214
   macro avg       0.73      0.74      0.74       214
weighted avg       0.80      0.79      0.79       214

A acurácia é  0.7897196261682243


# Random Forest



In [54]:
#Preparar dados para serem divididos entre treinamento e teste
X= input
y= sas
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=123)

In [55]:
#Realizadno a classificação random forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,
                             max_depth=5,
                             random_state=123,
                             n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_jobs=-1, random_state=123)

In [58]:
y_train_true = y_train
y_train_pred = clf.predict(X_train)
y_test_true = y_test
y_test_pred = clf.predict(X_test)


print(f"Acurácia de Treino: {round(accuracy_score(y_train_true, y_train_pred), 2)}")
print(f"Acurácia de Teste: {round(accuracy_score(y_test_true, y_test_pred), 2)}")

print(classification_report(train_y, train_est_y)) # mostra relatório

Acurácia de Treino: 0.87
Acurácia de Teste: 0.68
              precision    recall  f1-score   support

         0.0       0.87      0.84      0.85       157
         1.0       0.60      0.65      0.62        57

    accuracy                           0.79       214
   macro avg       0.73      0.74      0.74       214
weighted avg       0.80      0.79      0.79       214

